In [18]:
import numpy as np
import matplotlib.pyplot as plt
import struct,os
from array import array as pyarray
from numpy import append, array, int8, uint8, zeros

def load_mnist(image_file, label_file):
    digits=np.arange(10)

    fname_image = os.path.join("../"+image_file)
    fname_label = os.path.join("../"+label_file)

    flbl = open(fname_label, 'rb')
    magic_nr, size = struct.unpack(">II", flbl.read(8))
    lbl = pyarray("b", flbl.read())
    flbl.close()

    fimg = open(fname_image, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", fimg.read(16))
    img = pyarray("B", fimg.read())
    fimg.close()

    ind = [ k for k in range(size) if lbl[k] in digits ]
    N = len(ind)

    images = zeros((N, rows*cols), dtype=uint8)
    labels = zeros((N, 1), dtype=int8)
    for i in range(len(ind)):
        images[i] = array(img[ ind[i]*rows*cols : (ind[i]+1)*rows*cols ]).reshape((1, rows*cols))
        labels[i] = lbl[ind[i]]

    return images, labels


train_image, train_label = load_mnist("train-images.idx3-ubyte", "train-labels.idx1-ubyte")
test_image, test_label = load_mnist("t10k-images.idx3-ubyte", "t10k-labels.idx1-ubyte")

# naive Bayes

In [3]:

nbres=[]
label_type=10
D=[[] for _ in range(label_type)]
for i in range(train_image.shape[0]):
    D[int(train_label[i])].append(train_image[i])
pc=[len(D[i])/train_label.shape[0] for i in range(label_type)]
xc=[[[0 for _ in range(256)] for _ in range(train_image.shape[1])] for _ in range(label_type)]
for c in range(label_type):
    for x in D[c]:
        for i in range(x.shape[0]):
            xc[c][i][x[i]]+=1
for i in range(test_image.shape[0]):
    rt=np.array([np.log(pc[j]) for j in range(10)])
    x=test_image[i]
    for j in range(x.shape[0]):
        for c in range(label_type):
            rt[c]+=np.log(xc[c][j][x[j]]/len(D[c]) if xc[c][j][x[j]]>0 else (xc[c][j][x[j]]+1)/(len(D[c])+256))
    nbres.append(rt)

In [16]:
print(len(nbres))

10000


# Neural Networks (with hidden layer size 27*27)

In [17]:
import torch
db=torch.load("../PyTorchNN/trained_model_tensor2.pt")

In [12]:
v=db["v"].to('cpu')
gamma=db["gamma"].to('cpu')
w=db["w"].to('cpu')
theta=db["theta"].to('cpu')
m = torch.nn.Sigmoid()
def predict(img):
    return m((m((img @ v)-gamma) @ w)-theta)

In [15]:
Y1=predict(torch.from_numpy(test_image).float())
Y1.shape

torch.Size([10000, 10])

# Neural Networks (with hidden layer size 400)

In [21]:
from scipy.special import expit
npd=np.load("../NeuralNetworks/trained.npz")
gamma2=npd["v_gamma"]
w2=npd["m_w"]
theta2=npd["v_theta"]
v2=npd["m_v"]
def pred2(img):
    return expit(expit(img.dot(v2)-gamma2).dot(w2)-theta2)

In [22]:
Y2=pred2(test_image)
Y2.shape

(10000, 10)

In [23]:
err=[0 for _ in range(3)]
for i in range(test_image.shape[0]):
    truth=int(test_label[i])
    if(np.argmax(nbres[i])!=truth):
        err[0]+=1
    if(torch.argmax(Y1[i])!=truth):
        err[1]+=1
    if(np.argmax(Y2[i])!=truth):
        err[2]+=1
print("accuracy is %.10lf,%.10lf,%.10lf separately:"%(1-err[0]/10000,1-err[1]/10000,1-err[2]/10000))


accuracy is 0.8401000000,0.9141000000,0.8633000000 separately:


# 三个个体学习器平均加权

In [25]:
tote=0
for i in range(test_image.shape[0]):
    truth=int(test_label[i])
    accu=np.zeros(10)
    
    sm=np.sum(nbres[i]) # 和归一化
    accu+=(nbres[i]/sm)

    sm=torch.sum(Y1[i]).item()
    for j in range(10):
        accu[j]+=Y1[i][j]/sm
    
    sm=np.sum(Y2[i])
    accu+=(Y2[i]/sm)

    if(np.argmax(accu)!=truth):
        tote+=1
print(1-tote/test_image.shape[0])

0.9185


# 随便设置个加权（按准确率）

In [27]:
tote=0
for i in range(test_image.shape[0]):
    truth=int(test_label[i])
    accu=np.zeros(10)
    
    sm=np.sum(nbres[i])
    accu+=(nbres[i]/sm*0.8401)

    sm=torch.sum(Y1[i]).item()
    for j in range(10):
        accu[j]+=Y1[i][j]/sm*0.9141
    
    sm=np.sum(Y2[i])
    accu+=(Y2[i]/sm*0.8633)

    if(np.argmax(accu)!=truth):
        tote+=1
print(1-tote/test_image.shape[0])

0.919


# 多数投票

In [26]:
tote=0
for i in range(test_image.shape[0]):
    truth=int(test_label[i])
    tab=np.zeros(10)

    tab[np.argmax(nbres[i])]+=1

    torchres=torch.argmax(Y1[i])
    tab[torchres]+=1

    tab[np.argmax(Y2[i])]+=1

    predval=np.argmax(tab)
    if(tab[predval]==1): # 各不相同，按最精确的分类器的结果
        predval=torchres

    if(predval!=truth):
        tote+=1
    
print(1-tote/test_image.shape[0])

0.914
